# Purpose:

Make outputs for the Zachry Brenton Sorghum paper

# Pre-amble

In [ ]:
setwd("/scratch/")

Make a directory and get the data

In [ ]:
dir.create("sorghum/")
system("aws s3 sync s3://ddpsc-baxterlab-data/panvar/panvar_mwe/raw_inputs/sorghum/ sorghum/")

In [ ]:
system2("rclone copy dandrive:WIP/panvar/sorghum/BAP_WSC/sorghum_BAP_WSC_gwas.tsv .")

# Dependencies

In [ ]:
library(devtools)

Install `panvar`

In [ ]:
load_all("~/repos/devel_panvar/")

If you need to rebuild the library for `rmd`.

The order is important here.

In [ ]:
setwd("~/repos/devel_panvar/")

In [ ]:
devtools::document()

In [ ]:
devtools::build()

In [ ]:
devtools::build_rmd("vignettes/panvaR.Rmd")

# Inputs

# Scratch

In [ ]:
gwas_table <- panvar_gwas(
    genotype_data = "sorghum/BAP_376_Chrall_imputed_annotated.vcf.gz",
    phentotype_path = "sorghum/BAP_WSC_pheno.tsv"
)

In [ ]:
gwas_table %>% 
    fwrite("sorghum_BAP_WSC_gwas.tsv", sep = "\t", col.names = TRUE)

In [ ]:
gwas_table %>% 
    filter(CHROM == 4) %>%
    filter(Pvalues > 18)

In [ ]:
test_run <- panvar_func(
    phenotype_data_path = "sorghum/BAP_WSC_pheno.tsv",
    vcf_file_path = "sorghum/BAP_376_Chrall_imputed_annotated.vcf.gz",
    chrom = "Chr04",
    bp = 66529675,
    all.impacts = FALSE,
    r2_threshold = 0.8
)

In [ ]:
test_run2 <- panvar_func(
    phenotype_data_path = "sorghum/BAP_WSC_pheno.tsv",
    vcf_file_path = "sorghum/BAP_376_Chrall_imputed_annotated.vcf.gz",
    chrom = "Chr04",
    bp = 17548900,
    all.impacts = TRUE,
    r2_threshold = 0.8
)

Error
```txt
Error in `left_join()`:
! Can't join `x$CHROM` with `y$CHROM` due to incompatible types.
ℹ `x$CHROM` is a <character>.
ℹ `y$CHROM` is a <double>.
```

In [ ]:
phenotype_data_path = "sorghum/BAP_WSC_pheno.tsv"
vcf_file_path = "sorghum/BAP_376_Chrall_imputed_annotated.vcf.gz"
chrom = "Chr04"
bp = 17548900
all.impacts = TRUE

In [ ]:
window = 500000
missing_rate = 0.10
maf = 0.05
r2_threshold = 0.3

In [ ]:
gwas_table <- fread("sorghum_BAP_WSC_gwas.tsv")

In [ ]:
window_bp <- window_unit_func(window)

In [ ]:
in_plink_format <- vcf_to_plink2(vcf_file_path)

In [ ]:
cleaned_up <- bed_file_clean_up(in_plink_format$bed, maf = maf, missing_rate = missing_rate)

In [ ]:
subset_genotype_data <- subset_around_tag(cleaned_up,chrom = chrom, bp = bp, window = window_bp)

In [ ]:
table <- ld_filtered_snp_list(subset_genotype_data,chrom = chrom, bp = bp, r2_threshold = r2_threshold)

In [ ]:
ld_filtered_snp_list

In [ ]:
return_snplist_for_bp

In [ ]:
ld_table <- ld_table_maker(table)

In [ ]:
keep_snp_list <- snps_to_keep(table)

In [ ]:
plink2_bcf_dictionary <- plink2_bcftools_chroms_dictionary(vcf_file_path,in_plink_format$bim)

In [ ]:
if(!is.null(plink2_bcf_dictionary)){
		ld_table_checked <- apply_dict(plink2_bcf_dictionary, ld_table)

		snp_keep_list_checked <- apply_dict(plink2_bcf_dictionary, keep_snp_list)

        gwas_table_dicted <- apply_dict(plink2_bcf_dictionary, gwas_table)
	} else{
		
		ld_table_checked <-  ld_table

		snp_keep_list_checked <- keep_snp_list

        gwas_table_dicted <- gwas_table
	}

In [ ]:
keep_table_path <- keep_table_sanitizer(snp_keep_list_checked)

In [ ]:
keep_table_path

In [ ]:
filtered_vcf_table <- filter_vcf_file(vcf_file_path = vcf_file_path, keep_table_path)

In [ ]:
split_table_path <- split_vcf_eff(filtered_vcf_table)

In [ ]:
snpeff_table <- execute_snpsift(split_table_path)

In [ ]:
snpsift_table <- snpeff_table$table

In [ ]:
all.impacts = TRUE
if(all.impacts){
        snpsift_table_impacts <- snpsift_table
    } else {
        snpsift_table_impacts <- snpsift_table %>% 
            filter(IMPACT %in% c("HIGH","MODERATE") | BP == bp ) # The OR condition lets us retain the tag SNP which might be dropped if the IMPACT factor is not HIGH or MODERATE
    }

In [ ]:
pvalues_impact_ld_table <- snpsift_table_impacts %>%
        left_join(gwas_table_dicted, by = c("CHROM","BP")) %>%
        left_join(ld_table_checked, by = c("CHROM","BP"))

In [ ]:
    pvalues_impact_ld_colors_table <- pvalues_impact_ld_table %>% mutate(
        Type = case_when(
            BP == bp ~ "tag_snp",
            BP != bp ~ "Candidate"
        )
    )

In [ ]:
overall_weight_func(pvalues_impact_ld_colors_table, bp = bp) %>% 
    filter(IMPACT %in% c("HIGH","MODERATE"))

In [ ]:
test_run$table %>%
    filter(IMPACT != "MODIFIER") %>%
    filter(IMPACT != "LOW") %>%
    filter(LD >= 0.8)

In [ ]:
library(qqman)

In [ ]:
jpeg("~/in_progress/manhattan.jped", height = 1200, width = 1800)
manhattan(gwas_table,chr="CHROM", bp = "BP", p = "unlogged", snp = "snp", suggestiveline = FALSE, genomewideline = 11.7, cex.lab = 1.8, cex.axis = 1.8,mgp = c(2.5, 1, 0),ylab = "-log10(p)",logp = TRUE)
dev.off()

In [ ]:
manhattan(gwas_table,chr="CHROM", bp = "BP", p = "Pvalues", snp = "snp", suggestiveline = FALSE, genomewideline = 6.7, cex.lab = 1.8, cex.axis = 1.8,mgp = c(2.5, 1, 0),ylab = "-log10(p)")

In [ ]:
log10((10^-5)/nrow(gwas_table))

In [ ]:
gwas_table$unlogged = 1/10^(gwas_table$Pvalues)

In [ ]:
manhattan(gwas_table,chr="CHROM", bp = "BP", p = "unlogged", snp = "snp", suggestiveline = FALSE, genomewideline = 11.7, cex.lab = 1.8, cex.axis = 1.8,mgp = c(2.5, 1, 0),ylab = "-log10(p)",logp = TRUE)

In [ ]:
-log10((2.838*10e-7)/28.6529425104887)

In [ ]:
nrow(gwas_table)

In [ ]:
log10(2.838*10e-7 * 5053806)

In [ ]:
-log10(0.01/5053806)

In [ ]:
?manhattan